<a href="https://colab.research.google.com/github/augustompm/course-multiobjective-optimization/blob/main/multiobjetive_optimization_c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-hq3dyuqt
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-hq3dyuqt
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 801584cceb559adc54e828ebe9b385c5f53fe70f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-1o_3i0__/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [3]:
%load_ext nvcc4jupyter



Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpwmaiywlb".


In [4]:
%%cuda
//Olá mundo imprimindo de 1 até 10
#include <iostream>
int main() {
    for (int i = 1; i <= 10; ++i) {
        std::cout << i << " ";
    }
    std::cout << std::endl;
    return 0;
}


1 2 3 4 5 6 7 8 9 10 



In [20]:
%%cuda
#include <iostream>
#include <vector>
#include <cstdlib>
#include <ctime>

using namespace std;

// armazenar os dados dos itens
struct Item {
    int peso;
    int valor;
};

// solução aleatória dentro de um limite de tempo
vector<bool> heuristicaAleatoria(const vector<Item>& itens, int capacidade, int tempoLimite) {
    vector<bool> solucao(itens.size(), false); // Solução inicial (vazia)
    int pesoAtual = 0; // Peso total na mochila

    // Define o tempo de início
    time_t inicio = time(0);

    // adicionar itens aleatoriamente até o tempo limite
    while (difftime(time(0), inicio) < tempoLimite) {
        int indice = rand() % itens.size(); // Seleciona um item aleatório

        // verifica se o item pode ser adicionado
        if (!solucao[indice] && pesoAtual + itens[indice].peso <= capacidade) {
            solucao[indice] = true; // Adiciona o item à solução
            pesoAtual += itens[indice].peso; // Atualiza o peso total
            cout << "Item " << indice + 1 << " adicionado! Peso: " << itens[indice].peso << ", Valor: " << itens[indice].valor << endl;
        }
    }

    return solucao;
}

int calcularFO(const vector<Item>& itens, const vector<bool>& solucao) {
    int valorTotal = 0;
    for (int i = 0; i < itens.size(); ++i) {
        if (solucao[i]) {
            valorTotal += itens[i].valor;
        }
    }
    return valorTotal;
}

void imprimirSolucao(const vector<Item>& itens, const vector<bool>& solucao) {
    cout << "\nItens escolhidos para a mochila:\n";
    for (int i = 0; i < itens.size(); ++i) {
        if (solucao[i]) {
            cout << "Item " << i + 1 << " (Peso: " << itens[i].peso << ", Valor: " << itens[i].valor << ")\n";
        }
    }
    cout << "\nFunção Objetivo (Valor total dos itens): " << calcularFO(itens, solucao) << endl;
}

int main() {
    // semente aleatória básica
    srand(time(0));

    // itens e capacidade da mochila
    int capacidade = 10;
    vector<Item> itens = {
        {1, 1},  // Item 1: Peso 1, Valor 1
        {1, 2},  // Item 2: Peso 1, Valor 2
        {1, 3},  // Item 3: Peso 1, Valor 3
        {5, 7},  // Item 4: Peso 5, Valor 7
        {5, 8}   // Item 5: Peso 5, Valor 8
    };

    int tempoLimite = 2; // 2 segundos

    // Gera uma solução aleatória
    vector<bool> solucao = heuristicaAleatoria(itens, capacidade, tempoLimite);

    // Imprime a solução gerada e o valor total (Função Objetivo)
    imprimirSolucao(itens, solucao);

    return 0;
}


Item 1 adicionado! Peso: 1, Valor: 1
Item 5 adicionado! Peso: 5, Valor: 8
Item 2 adicionado! Peso: 1, Valor: 2
Item 3 adicionado! Peso: 1, Valor: 3

Itens escolhidos para a mochila:
Item 1 (Peso: 1, Valor: 1)
Item 2 (Peso: 1, Valor: 2)
Item 3 (Peso: 1, Valor: 3)
Item 5 (Peso: 5, Valor: 8)

Função Objetivo (Valor total dos itens): 14

